In [ ]:
#!pip install pandas
#!pip install matplotlib
#!python -m pip install tensorflow
#!pip install nvidia-tensorrt
#!pip install sagemaker

In [1]:
import pandas as pd
import numpy as np
import ast
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

2023-02-19 23:49:33.360815: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-19 23:49:33.362157: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-19 23:49:33.362165: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


In [2]:
sampling_rate=100

# Get the size of each sample
input_size = (1000,12,1)

In [3]:
# Train
X_train = np.load('X_train.npy')
y_train = np.load('y_train.npy')
# Test
X_test = np.load('X_test.npy')
y_test = np.load('y_test.npy')

In [4]:
X_train_norm_1 = (X_train - np.min(X_train))/np.ptp(X_train) # [0, 1]
X_train_norm_2 = 2.*(X_train - np.min(X_train))/np.ptp(X_train)-1 # [-1,1]

In [5]:
inputs = keras.Input(shape=input_size)

In [6]:
x = keras.layers.Conv2D(64, (7, 7), strides=(2, 2), padding='same')(inputs)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Activation('sigmoid')(x)

In [7]:
def conv_block(input_tensor, filters, strides):
    x = keras.layers.Conv2D(filters, (1, 1), strides=strides)(input_tensor)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation('sigmoid')(x)

    x = keras.layers.Conv2D(filters, (3, 3), padding='same')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation('sigmoid')(x)

    x = keras.layers.Conv2D(4 * filters, (1, 1))(x)
    x = keras.layers.BatchNormalization()(x)

    shortcut = keras.layers.Conv2D(4 * filters, (1, 1), strides=strides)(input_tensor)
    shortcut = keras.layers.BatchNormalization()(shortcut)

    x = keras.layers.add([x, shortcut])
    x = keras.layers.Activation('sigmoid')(x)
    return x

def identity_block(input_tensor, filters):
    x = keras.layers.Conv2D(filters, (1, 1))(input_tensor)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation('sigmoid')(x)

    x = keras.layers.Conv2D(filters, (3, 3), padding='same')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation('sigmoid')(x)

    x = keras.layers.Conv2D(4 * filters, (1, 1))(x)
    x = keras.layers.BatchNormalization()(x)

    x = keras.layers.add([x, input_tensor])
    x = keras.layers.Activation('sigmoid')(x)
    return x


x = conv_block(x, 64, strides=(1, 1))
x = identity_block(x, 64)
x = identity_block(x, 64)

x = conv_block(x, 128, strides=(2, 2))
x = identity_block(x, 128)
x = identity_block(x, 128)
x = identity_block(x, 128)

x = conv_block(x, 256, strides=(2, 2))
x = identity_block(x, 256)
x = identity_block(x, 256)
x = identity_block(x, 256)
x = identity_block(x, 256)
x = identity_block(x, 256)

x = conv_block(x, 512, strides=(2,2))
x = identity_block(x, 512)
x = identity_block(x, 512)

x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dense(5, activation='sigmoid')(x)

In [11]:
model = keras.Model(inputs=inputs, outputs=x, name='resnet2')
model.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy'])

In [12]:
X_test_norm_1 = (X_test - np.min(X_train))/np.ptp(X_train) # [0, 1]

In [ ]:
history = model.fit(x=X_train_norm_1, y=y_train,
                    batch_size=32,
                    epochs=60, validation_data=(X_test_norm_1, y_test))

Epoch 1/60
613/613 [==============================] - 250s 359ms/step - loss: 0.6065 - accuracy: 0.4821 - val_loss: 0.7831 - val_accuracy: 0.1720
Epoch 2/60
613/613 [==============================] - 224s 365ms/step - loss: 0.4470 - accuracy: 0.5634 - val_loss: 0.4731 - val_accuracy: 0.4682
Epoch 3/60
613/613 [==============================] - 222s 363ms/step - loss: 0.3977 - accuracy: 0.6089 - val_loss: 0.4854 - val_accuracy: 0.5350
Epoch 4/60
613/613 [==============================] - 221s 361ms/step - loss: 0.3743 - accuracy: 0.6305 - val_loss: 0.3916 - val_accuracy: 0.5942
Epoch 5/60
613/613 [==============================] - 221s 361ms/step - loss: 0.3595 - accuracy: 0.6488 - val_loss: 0.4861 - val_accuracy: 0.5523
Epoch 6/60
613/613 [==============================] - 222s 362ms/step - loss: 0.3492 - accuracy: 0.6569 - val_loss: 0.4540 - val_accuracy: 0.5537
Epoch 7/60
613/613 [==============================] - 222s 362ms/step - loss: 0.3403 - accuracy: 0.6677 - val_loss: 0.4567 -

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Precisión del modelo')
plt.ylabel('Precisión')
plt.xlabel('Época')
plt.legend(['Entrenamiento', 'Validación'], loc='upper left')
plt.show()

In [ ]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
#X_test_norm = 2.*(X_test - np.min(X_train))/np.ptp(X_train)-1 # [-1,1]
X_test_norm = (X_test - np.min(X_train))/np.ptp(X_train) # [0, 1]
results = model.evaluate(X_test_norm, y_test)
print("test loss, test acc:", results)

In [ ]:
model.save("resnet_model_2")